In [1]:
from itertools import chain
from collections import Counter
import operator

# Define All Function

In [2]:
def get_data(filename):
    """Fungsi untuk melakukan load data pada file .tsv maupun .txt
    
    File yang digunakan yaitu file .tsv maupun file yang mampu dibuka
    menggunakan teks editor. Dalam file tersebut terdapat tag pembuka <kalimat>
    dan diakhiri tag penutup </kalimat> yang berfungsi sebagai penanda bahwa
    entitas tersebut termasuk ke dalam satu kalimat.
    
    Args:
        filename: string dari nama file yang akan diload datanya.
        
    Return:
        list dari kata-kata dan tags dengan index yang menunjukkan posisi kalimat tersebut.
    """
    
    # Load data dan buka sebagai file
    sentences = []
    tags = []
    with open(filename) as file:
        contents = file.readlines()

    # Hapus karakter \n yang tidak dibutuhkan
    contents = [content.strip() for content in contents]
    idx = 0
    while idx < len(contents):
        word = []
        tag = []
        # looping sampai menemukan pattern dengan awalan </kalimat
        while not contents[idx].startswith('</kalimat'):
            # kondisi jika menemukan sebuah data yang tidak memiliki awalan <kalimat
            if not contents[idx].startswith('<kalimat'):
                temp_word, temp_tag = contents[idx].split("\t")
                word.append(temp_word.lower())
                tag.append(temp_tag)
            idx += 1
        sentences.append(word)
        tags.append(tag)
        idx += 2
        
    return sentences, tags

In [3]:
def flatten(multi_list):
    """Mengembalikan list multi dimensi ke dalam list satu dimensi.
    
    Input list yang masuk di proses menggunakan method from_iterable dari
    package itertools dan mengembalikan object berupa generator yang iterable.
    
    Args:
        multi_list: list multi dimensi.
    
    Return:
        list satu dimensi yang nantinya digunakan untuk pemrosesan lebih lanjut.
    """
    
    return chain.from_iterable(multi_list)

In [4]:
def get_bigram(dataset):
    """Fungsi untuk membuat bigram dari list.
    
    Input list berisi string yang nantinya digenerate bigram dan mengembalikan
    objek berupa dictionary python.
    
    Args:
        dataset: list satu dimensi.
        
    Return:
        dictionary yang memuat bigram dan banyaknya bigram pada list tersebut.
    """
    
    bigrams = []
    for i in range(len(dataset)-1):
        bigrams.append((dataset[i], dataset[i+1]))
    count_bigrams = Counter(bigrams)
    
    return bigrams, dict(count_bigrams)

In [5]:
def get_prob_transition(dict_bigrams):
    """Fungsi untuk menghitung probabilitas dari dictionary perhitungan banyaknya muncul pasangan tag
    
    Tag diambil dan dihitung berdasarkan banyak kemunculan dibagi dengan total keseluruhan tag
    yang muncul.
    
    Args:
        dict_bigrams: dictionary berisi pasangan tag dan banyaknya kemunculan tag
    
    Return:
        tabel transisi yang berisi probabilitas kemunculan tag dan tag berikutnya
    
    """
    total_cnt = float(sum(dict_bigrams.values()))
    probs = []
    keys = []
    
    for key, value in dict_bigrams.items():
        probs.append(value / total_cnt)
        keys.append(key)
    
    probs_transition = dict(zip(keys,probs))
    return probs_transition

In [6]:
def get_emission_table(words, tags):
    # Membuat representasi tabel emission probability dari HMM
    hidden_state = {}
    for word in words:
        # Jika word belum pernah ditemui maka akan digenerate
        # tagset berserta probabilitasnya
        if word not in hidden_state.keys():
            word_tags = []
            for idx, wrd in enumerate(words):
                if wrd == word :
                    word_tags.append(tags[idx])
            # Membuat dictionary berisi semua tagset dari sebauh word
            # pada korpus. Berserta jumlah kemunculan setiap tagsetnya
            tag_count = {}
            for tag in word_tags:
                if tag not in tag_count.keys():
                    tag_count[tag] = 1
                else:
                    tag_count[tag] += 1
            total = 0
            # Mengubah jumlah kemunculan menjadi probabilitas
            tag_prob = {}
            for count in tag_count.values():
                total += count
            for tagset in tag_count.keys():
                tag_prob[tagset] = tag_count[tagset]/total
            hidden_state[word] = tag_prob
        else:
            # Jika word sudah pernah ditemui maka akan dilewati
            continue
    return(hidden_state)

# Load and Separate Dataset

Load dan pisahkan dataset sebanyak 1000 data train dan 20 data test

In [7]:
filename = '../data/corpus.tsv'
sentences, tags = get_data(filename)

In [8]:
# mengambil 1020 data awal
data_train, data_test, tags_train, tags_test = sentences[:1000], sentences[1000:1020], tags[:1000], tags[1000:1020]

In [9]:
data_train = list(flatten(data_train))
tags_train = list(flatten(tags_train))

Tabel transisi dibuat berdasarkan data train yang telah diambil pada corpus

In [10]:
bigrams, count_bigrams = get_bigram(tags_train)
transition_table = get_prob_transition(count_bigrams)

Tabel emisi dibuat berdasarkan data train yang telah diambil pada corpus

In [11]:
hidden_table = get_emission_table(data_train, tags_train)

# Baseline Model

In [39]:
def predict_common_tag(data, emmision_table):
    # dengan menggunakan hidden table untuk membangun baseline model berdasarkan corpus
    sentences_predict_tags = []
    for sentence in data:
        predict_tags = []
        for word in sentence:
            if word not in emmision_table.keys():
                predict_tags.append('NN')
            else:
                most_common_tag = max(emmision_table[word].items(), key=operator.itemgetter(1))[0]
                predict_tags.append(most_common_tag)
        
        sentences_predict_tags.append(predict_tags)
    
    return sentences_predict_tags

In [40]:
predict_tags = predict_common_tag(data_test, hidden_table)